In [1]:
import os
import pandas as pd

In [2]:
# Define the DSM file and the list of image files
input_dir = "./input"
dsm = f"{input_dir}/coata_dsm.tif"
images_dir = f"{input_dir}/images"

# List all jpg images in the images directory
images = [f"{images_dir}/{image}" for image in os.listdir(images_dir) if image.endswith((".jpg", ".JPG"))]


In [3]:

# Construct the command to run the script with all images
command = f"python ./src/obtain_location_from_center.py {dsm} " + " ".join(images)

# Execute the command
get_ipython().system(command)

ERROR with ./input/images/DJI_20240430105726_0098_D.JPG, make HASSELBLAD not compatible with this program!
skipping ./input/images/DJI_20240430105726_0098_D.JPG
ERROR with ./input/images/DJI_20240430112812_0138_D.JPG, make HASSELBLAD not compatible with this program!
skipping ./input/images/DJI_20240430112812_0138_D.JPG
ERROR: resolveTarget ran out of bounds at -1.4846, -59.7882, 81.7538m
ERROR: Please ensure target location is within GeoTIFF dataset bounds
ERROR with ./input/images/DJI_20240430104720_0080_D.JPG, make HASSELBLAD not compatible with this program!
skipping ./input/images/DJI_20240430104720_0080_D.JPG
ERROR with ./input/images/DJI_20240430112135_0126_D.JPG, make HASSELBLAD not compatible with this program!
skipping ./input/images/DJI_20240430112135_0126_D.JPG
ERROR with ./input/images/DJI_20240430104503_0075_D.JPG, make HASSELBLAD not compatible with this program!
skipping ./input/images/DJI_20240430104503_0075_D.JPG
ERROR with ./input/images/DJI_20240430105741_0099_D.JPG

In [4]:
# Parse the .ATHENA files and convert to a csv
data = []
directory = './output'
for filename in os.listdir(directory):
    if filename.endswith(".ATHENA"):
        clean_filename = filename.replace(".ATHENA", "")
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as file:
            lines = file.readlines()
            lat = float(lines[0].strip())
            lon = float(lines[1].strip())
            alt = float(lines[2].strip())
            data.append({'file_name': clean_filename, 'lat': lat, 'lon': lon, 'alt': alt})
df = pd.DataFrame(data, columns=['file_name', 'lat', 'lon', 'alt'])
# print(df)
df.to_csv('./output/output.csv', index=False)

In [5]:
# save as geopackage

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Load the CSV file
df = pd.read_csv('./output/output.csv')

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(
    df, 
    geometry=[Point(xy) for xy in zip(df.lon, df.lat)],
    crs="EPSG:4326"  # Assuming the coordinates are in WGS84; adjust if different
)

# Save to GeoPackage
gdf.to_file("output/points.gpkg", driver="GPKG")